# Summarization

This notebook walks through how to use LangChain for summarization over a list of documents. It covers three different chain types: `stuff`, `map_reduce`, and `refine`. For a more in depth explanation of what these chain types are, see [here](../../explanation/combine_docs.md).

### Prepare Data
First we prepare the data. For this example we create multiple documents from one long one, but these documents could be fetched in any manner (the point of this notebook to highlight what to do AFTER you fetch the documents).

In [23]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain

llm = OpenAI(temperature=0)


text_splitter = CharacterTextSplitter(chunk_size=1000, separator="\n")


In [24]:
with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

In [25]:
from langchain.docstore.document import Document

In [26]:
docs = [Document(page_content=t) for t in texts]

In [6]:
from langchain.chains.summarize import load_summarize_chain

### The `stuff` Chain

This sections shows results of using the `stuff` Chain to do summarization.

In [6]:
chain = load_summarize_chain(llm, chain_type="stuff")

In [7]:
chain.run(docs)

' In his speech, President Biden addressed the ongoing conflict between Russia and Ukraine, and the need for the United States and its allies to stand with Ukraine. He also discussed the American Rescue Plan, the Bipartisan Infrastructure Law, and the Bipartisan Innovation Act, which will help to create jobs, modernize infrastructure, and level the playing field with China. He also emphasized the importance of buying American products to support American jobs.'

### The `map_reduce` Chain

This sections shows results of using the `map_reduce` Chain to do summarization.

In [36]:
class CustomLLM(OpenAI):
    def _generate(
        self, prompts, stop = None
    ):
        for p in prompts:
            if self.get_num_tokens(p) > 600:
                raise ValueError
        return super()._generate(prompts, stop=stop)

In [41]:
llm = CustomLLM(temperature=0, model_name='text-curie-001')
chain = load_summarize_chain(llm, chain_type="map_reduce")

In [42]:
chain.run(input_documents=docs, token_max=400)

ValueError: A single document was so long it could not be combined with another document, we cannot handle this.

### The `refine` Chain

This sections shows results of using the `refine` Chain to do summarization.

In [10]:
chain = load_summarize_chain(llm, chain_type="refine")

In [11]:
chain.run(docs)

"\nIn this speech, the speaker addresses the American people and their allies, discussing the recent aggression of Russia's Vladimir Putin in Ukraine. The speaker outlines the actions taken by the United States and its allies to hold Putin accountable, including economic sanctions, cutting off access to technology, and seizing the assets of Russian oligarchs. The speaker also announces the closing of American airspace to Russian flights, further isolating Russia and adding an additional squeeze on their economy. The Russian stock market has lost 40% of its value and trading remains suspended. Together with our allies, the United States is providing military, economic, and humanitarian assistance to Ukraine, and has mobilized forces to protect NATO countries. The speaker also announces the release of 60 million barrels of oil from reserves around the world, with the United States releasing 30 million barrels from its own Strategic Petroleum Reserve. The speaker emphasizes that the Unite